# AI & Law - Challenge 1 - Liability

## Case-Study

<div dir="auto">
ביום 1.1.2020 בשעה 19:47, מספר שניות לאחר שהשתלב בצומת, פגע רכב אוטונומי מתוצרת "Play" (יצרנית רכבים המפתחת ומשווקת רכבים אוטונומיים לצד רכבים קלאסיים) בהולך הרגל רוג'ר. 
במושב הנהג ישב בראד, אשר בעת התאונה היה עסוק במכשיר הטלפון שלו. מספר דקות לאחר התאונה, בעת חקירתו על ידי שוטרת תנועה, העיד בראד שהוא ראה את רוג'ר מתהלך באמצע הכביש אבל הניח שהרכב יעצור ולא יפגע בו. 
רוג'ר נזקק לטיפול רפואי יקר, והוא מעוניין לתבוע (במדינת פלורידה) בשל הנזקים שנגרמו לו. 
</div>


## Prerequisite Object Detection & mAP

1. [Intro to object localization and detection video](https://youtu.be/GSwYGkTfOKk) (12 minutes) by Andrew Ng from deeplearning.ai
2. [Metrics for object detection](https://github.com/rafaelpadilla/Object-Detection-Metrics/blob/master/README.md) - from "Important definitions" until "How to use this project" (without!)
3. [mAP](https://medium.com/@jonathan_hui/map-mean-average-precision-for-object-detection-45c121a31173) - mean Average Precision

## Uncomment and Run Once!

In [ ]:
# Install dependencies

!pip install -r requirements.txt

In [ ]:
# Download trained model

!mkdir -p checkpoints

!wget --load-cookies /tmp/cookies.txt \
"https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1GZ4S2o5_ICtG3Ffl1X3eCD2KbK5k3Yxt' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1GZ4S2o5_ICtG3Ffl1X3eCD2KbK5k3Yxt" \
-O ./checkpoints/yolov3_ckpt_99.pth && rm -rf /tmp/cookies.txt

## Traffic Sign Detection

**Model:** YOLOv3 on PyTourch implemented by [Erik Linder-Norén](https://github.com/eriklindernoren/PyTorch-YOLOv3), GPL-3.0

**Dataset:** [Traffic Signs Dataset in YOLO format
](https://www.kaggle.com/valentynsichkar/traffic-signs-dataset-in-yolo-format) by [Valentyn Sichkar
](https://valentynsichkar.name/) (Originally: [GTSDB](http://benchmark.ini.rub.de/?section=gtsdb) (German Traffic Sign Detection Benchmark) by INI)

### Setup

In [ ]:
import numpy as np
import matplotlib.pylab as plt
from tqdm import tqdm, trange

from challenge import (load_model, load_class_names,
                       draw_detections,
                       draw_multiple_detections,
                       draw_evaluation,
                       detect_multi,
                       apply_augmentation, evaluate)

In [ ]:
model_def = 'config/yolov3-custom.cfg'
data_config = 'config/custom.data'
weights_path = 'checkpoints/yolov3_ckpt_99.pth'

Loading the model:

In [ ]:
model = load_model(weights_path, model_def, data_config)

All the traffic signs are divided into four groups, which are the lables that the model predcit:

In [ ]:
class_names = load_class_names(data_config)

class_names

You can find additional details about how each traffic signn is assigned to a label in [the dataset page in Kaggle](https://www.kaggle.com/valentynsichkar/traffic-signs-dataset-in-yolo-format) (Search for "Traffic Sins in this Dataset are grouped into four categories" section)

### Image from the accident
<small>Source: Google Street View</small>

`./data/custom/accident/accident-day.png`
![](./data/custom/accident/accident-day.png)

### Let's try to classify the two images using the Traffic Sign Detector

In [ ]:
# The function detect_multi with the argument `path_type='folder'`
# iterates over all the images in the `path` argument,
# it returns a list of paths as the first returned value
# and a list of images as 4-dim numpy array (NWHC) as the second returned value
accident_img_paths, accident_img_detections = detect_multi(model, class_names,
                                                           path= './data/custom/accident', path_type='folder')

# Draw the detections one after another
for p, d in zip(accident_img_paths, accident_img_detections):
    draw_detections(p, d)

### Training and Validation Datasets

The files `./data/custom/train.txt` and `./data/custom/valid.txt` holds the paths of the training and validation dataset imgages, respectively.


In [ ]:
!head ./data/custom/train.txt 

In [ ]:
!head ./data/custom/valid.txt 

We can evalauate the preformance of the model on the training dataset (remove the `report` argument to avoid printing). You can chang the path to the validataion dataset as well (`./data/custom/train.txt` → `./data/custom/valid.txt`).

In [ ]:
precision, recall, AP, f1, ap_class, mAP, evaluations_df = evaluate(
    model,
    './data/custom/train.txt',
    class_names,
    label_folder_path='data/custom/labels',
    report=True
)

There are two important variables:
1. `mAP`  holds the mean Average Precision
2. `evaluations_df` holds a pandas dataframe of the evaluation of each image

In [ ]:
mAP

In `evaluations_df` there are eight columns:
1. `path` - The path to the image
2. `tp` - For each prediction of the model, whehter it is true positive (1) or false positive (0)
3. `pred_confs` - The confidence score of the model to each of its prediction, in the same order of `tp`
4. `pred_labels` - The label that the model predited, in the same order of `tp`
5. `true_labels` - The true labels
6. `precision` - Precision of all the predictions
7. `recall` - Recall over all the predictions
8. `detections` - Bounding boxes of the predictions (primarly for internal use of some functions)

In [ ]:
evaluations_df.head()

With `draw_multiple_detections` we can draw evaluations (we use the method `sample` to get random images to explore, so you can run the next cell multiple times and get new images):

In [ ]:
draw_multiple_detections(evaluations_df.sample(4));

Note that you have also the text file `./data/custom/accident.txt` that contains the path of the two accident images:

In [ ]:
!cat ./data/custom/accident.txt

In [ ]:
# Your code here...

## Exploring Augmentations

To understand the impact of different augmentations on the model preformance, we can use the function `apply_augmentation` which create a copy of all the images by a paths from a text file (e.g., `./data/custom/valid.txt`), a augmentation function (e.g., `am.add_rain`) and parameters for each augmentaton (e.g., `rain_type='dizzle`, `drop_width=3`). The function returns the text file that contains all the paths to the new generated images.

See the complete documentation of the augmentation functions [**here**](https://github.com/UjjwalSaxena/Automold--Road-Augmentation-Library/blob/master/README.md). You don't need to import anything new, as all the functionality is in the already existing package `augmentation`.


In [ ]:
import augmentation.automold as am

In [ ]:
aug_rain_path = apply_augmentation('./data/custom/valid.txt',
                                   am.add_rain,
                                   rain_type='heavy',
                                   drop_width=1)

_, _, _, _, _, mAP,  df = evaluate(
    model,
    aug_rain_path,
    class_names,
    label_folder_path='data/custom/labels',
    progress=iter,
    report=True
)

In [ ]:
# HIDDEN

aug_gravel_path = apply_augmentation('./data/custom/valid.txt',
                                   am.add_gravel,
                                   no_of_patches=45)

_, _, _, _, _, mAP,  df = evaluate(
    model,
    aug_gravel_path,
    class_names,
    label_folder_path='data/custom/labels',
    progress=iter,
    report=True
)

In [ ]:
# HIDDEN

aug_rain_gravel_path = apply_augmentation(aug_rain_path,
                                   am.add_gravel,
                                   no_of_patches=45)

_, _, _, _, _, mAP,  df = evaluate(
    model,
    aug_rain_gravel_path,
    class_names,
    label_folder_path='data/custom/labels',
    progress=iter,
    report=True
)

In your analysis, focus on `am.add_rain` and `am.add_gravel`. We think about the latter as dirt on the camera.

In [ ]:
# HIDDEN

_, _, _, _, _, _, evaluations_df = evaluate(
    model,
    './data/custom/valid.txt',
    class_names,
    label_folder_path='data/custom/labels',
    report=False
)

aug_paths_files = {}

for rain_type in ['dizzle', 'heavy']:
    for drop_width in trange(2, 3):
        aug_paths_files[f'rain-{rain_type}-{drop_width}'] = apply_augmentation('./data/custom/valid.txt',
                                                                   am.add_rain,
                                                                   rain_type=rain_type,
                                                                   drop_width=drop_width)

for aug_tag, prev_aug_path in aug_paths_files.copy().items():
    for no_of_patches in trange(45, 55, 10):
        aug_paths_files[f'{aug_tag}_gravel-{no_of_patches}'] = apply_augmentation(prev_aug_path,
                                                                   am.add_gravel,
                                                                   no_of_patches=no_of_patches)
    
for no_of_patches in trange(45, 55, 10):
    aug_paths_files[f'gravel-{no_of_patches}'] = apply_augmentation('./data/custom/valid.txt',
                                                               am.add_gravel,
                                                               no_of_patches=no_of_patches)


mAPs = {}
evaluations_dfs = {}

for aug, path in aug_paths_files.items():
    print(f'### {aug} ###')
    _, _, _, _, _, mAP,  df = evaluate(
        model,
        path,
        class_names,
        label_folder_path='data/custom/labels',
        progress=iter,
        report=False
    )
    
    mAPs[aug] = mAP
    evaluations_dfs[aug] = df
    
mAPs

In [ ]:
# HIDDEN

from ipywidgets import interact, fixed

@interact(augmentation=list(tuple(evaluations_dfs.keys())),
          index=(0, len(evaluations_df)-1),
          mAPs=fixed(mAPs))
def explore_augmentation(index, augmentation, mAPs):
    _, axes = plt.subplots(1, 2, figsize=(15, 15))
    
    draw_evaluation(evaluations_dfs[augmentation].iloc[index], f'Augmented (mAP={mAPs[augmentation]}\n',
                    ax=axes[0]);
    draw_evaluation(evaluations_df.iloc[index], 'Original\n', ax=axes[1]);

    axes[0].axis('off');
    axes[1].axis('off');

In [ ]:
# HIDDEN

# Generate accident day image
# NOTE: Sometimes the classifer managed to work wall on the image with heavy rain 2,
#       so multiple runs are required for a FN

# Remove this to generate the accident images
raise RuntimeError

accident_rain_path = apply_augmentation('./data/custom/accident.txt',
                       am.add_rain,
                       rain_type='heavy',
                       drop_width=2)

accident_gravel_path = apply_augmentation('./data/custom/accident.txt',
                       am.add_gravel,
                       no_of_patches=45)


accident_rain_gravel_path = apply_augmentation(accident_rain_path,
                                               am.add_gravel,
                                               no_of_patches=45)

accident_img_paths, accident_img_detections = detect_multi(model, class_names,
                                                           'data/custom/accident--aug-add_rain_heavy-2', 'folder',
                                                            progress=iter)

for p, d in zip(accident_img_paths, accident_img_detections):
    draw_detections(p, d)

accident_img_paths, accident_img_detections = detect_multi(model, class_names,
                                                           'data/custom/accident--aug-add_gravel_45', 'folder',
                                                            progress=iter)

for p, d in zip(accident_img_paths, accident_img_detections):
    draw_detections(p, d)
    
accident_img_paths, accident_img_detections = detect_multi(model, class_names, './data/custom/accident--aug-add_rain_heavy-2--aug-add_gravel_45',
                                         path_type='folder', progress=iter)

for p, d in zip(accident_img_paths, accident_img_detections):
    draw_detections(p, d)